# Inside Airbnb Madrid: Neighbourhoods & Reviews Integration

Objective: Analyze `neighbourhoods.csv` and `reviews.csv`, clean them, and integrate with `calendar_clean` to create enriched datasets with review-derived features and spatial/administrative context.

Tasks:
- Task A: EDA on neighbourhoods and reviews (data quality, structure, join keys)
- Task B: Clean and standardize both datasets
- Task C: Integrate reviews (review-derived features) and neighbourhoods with calendar_clean
- Deliverables: Cleaned datasets, enriched calendar, integration strategy, web-map suggestions

---

## Section 0: Environment Setup & Library Versions

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Print library versions
print("=" * 80)
print("LIBRARY VERSIONS & ENVIRONMENT")
print("=" * 80)
print(f"Python: {sys.version}")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}\n")

# Configuration - RELATIVE PATHS (reproducible across machines)
# Assumes execution from project root directory OR from scripts/ subdirectory
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'scripts' else Path.cwd()
DATASET_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = DATASET_DIR / "processed"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project Root: {PROJECT_ROOT}")
print(f"Dataset Dir: {DATASET_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")
print(f"(All paths relative to current working directory)\n")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 10)

print("✓ Environment initialized")

LIBRARY VERSIONS & ENVIRONMENT
Python: 3.12.12 | packaged by conda-forge | (main, Jan 27 2026, 00:01:15) [Clang 19.1.7 ]
pandas: 3.0.0
numpy: 2.4.2

Project Root: /Users/virginiadimauro/Desktop/UNITN/Secondo Anno/Geospatial Analysis/geospatial-project
Dataset Dir: /Users/virginiadimauro/Desktop/UNITN/Secondo Anno/Geospatial Analysis/geospatial-project/data
Output Dir: /Users/virginiadimauro/Desktop/UNITN/Secondo Anno/Geospatial Analysis/geospatial-project/data/processed
(All paths relative to current working directory)

✓ Environment initialized


## Task A1: Neighbourhoods.csv EDA

Load and analyze the neighbourhoods dataset: structure, columns, data types, missing values, duplicates, and join potential.

In [14]:
# Load neighbourhoods.csv
neighbourhoods_file = DATASET_DIR / "neighbourhoods.csv"
print("=" * 80)
print("NEIGHBOURHOODS.CSV - EXPLORATORY DATA ANALYSIS")
print("=" * 80)

print(f"\n📁 File: {neighbourhoods_file.name}")
print(f"   Size: {neighbourhoods_file.stat().st_size / (1024**2):.2f} MB\n")

df_neighbourhoods = pd.read_csv(neighbourhoods_file)

print("SHAPE & COLUMNS:")
print(f"  Shape: {df_neighbourhoods.shape} (rows, columns)")
print(f"  Columns: {list(df_neighbourhoods.columns)}\n")

print("DATA TYPES:")
print(df_neighbourhoods.dtypes)
print()

print("HEAD (first 10 rows):")
print(df_neighbourhoods.head(10))
print()

print("TAIL (last 5 rows):")
print(df_neighbourhoods.tail(5))
print()

# Missing value analysis
print("MISSING VALUES (per column):")
missing_pct = (df_neighbourhoods.isnull().sum() / len(df_neighbourhoods) * 100)
for col, pct in missing_pct.items():
    print(f"  {col}: {df_neighbourhoods[col].isnull().sum():,} / {len(df_neighbourhoods):,} ({pct:.2f}%)")
print()

# Duplicates analysis
print("DUPLICATES ANALYSIS:")
full_row_dups = df_neighbourhoods.duplicated().sum()
print(f"  Full-row duplicates: {full_row_dups}")

if df_neighbourhoods.shape[1] > 0:
    for col in df_neighbourhoods.columns:
        col_dups = df_neighbourhoods[col].duplicated().sum()
        if col_dups > 0:
            print(f"  Duplicates in '{col}': {col_dups}")
print()

# Identify potential primary keys
print("PRIMARY KEY CANDIDATES:")
for col in df_neighbourhoods.columns:
    unique_count = df_neighbourhoods[col].nunique()
    is_pk_candidate = (unique_count == len(df_neighbourhoods)) and (df_neighbourhoods[col].isnull().sum() == 0)
    print(f"  {col}: {unique_count} unique values, {df_neighbourhoods[col].isnull().sum()} nulls → PK candidate: {is_pk_candidate}")
print()

# Data quality issues
print("DATA QUALITY CHECKS:")
for col in df_neighbourhoods.columns:
    if df_neighbourhoods[col].dtype == 'object':
        empty_str = (df_neighbourhoods[col] == '').sum()
        if empty_str > 0:
            print(f"  ⚠️  '{col}': {empty_str} empty strings")
        
        # Check for leading/trailing spaces
        has_spaces = df_neighbourhoods[col].str.strip().ne(df_neighbourhoods[col]).sum()
        if has_spaces > 0:
            print(f"  ⚠️  '{col}': {has_spaces} values with leading/trailing spaces")

print("\n✓ Neighbourhoods EDA complete")

NEIGHBOURHOODS.CSV - EXPLORATORY DATA ANALYSIS

📁 File: neighbourhoods.csv
   Size: 0.00 MB

SHAPE & COLUMNS:
  Shape: (128, 2) (rows, columns)
  Columns: ['neighbourhood_group', 'neighbourhood']

DATA TYPES:
neighbourhood_group    str
neighbourhood          str
dtype: object

HEAD (first 10 rows):
  neighbourhood_group               neighbourhood
0          Arganzuela                     Acacias
1          Arganzuela                      Atocha
2          Arganzuela                     Chopera
3          Arganzuela                    Delicias
4          Arganzuela                    Imperial
5          Arganzuela                     Legazpi
6          Arganzuela             Palos de Moguer
7             Barajas                  Aeropuerto
8             Barajas            Alameda de Osuna
9             Barajas  Casco Histórico de Barajas

TAIL (last 5 rows):
    neighbourhood_group  neighbourhood
123          Villaverde       Butarque
124          Villaverde    Los Angeles
125         

## Task A2: Reviews.csv EDA

Load and analyze the reviews dataset: structure, columns, data types, missing values, review date range, and feasibility for creating aggregated features.

In [15]:
# Load reviews.csv
reviews_file = DATASET_DIR / "reviews.csv"
print("\n" + "=" * 80)
print("REVIEWS.CSV - EXPLORATORY DATA ANALYSIS")
print("=" * 80)

print(f"\n📁 File: {reviews_file.name}")
print(f"   Size: {reviews_file.stat().st_size / (1024**2):.2f} MB\n")

# Load with string dtype for date to preserve format
df_reviews = pd.read_csv(reviews_file, dtype_backend='numpy_nullable')

print("SHAPE & COLUMNS:")
print(f"  Shape: {df_reviews.shape} (rows, columns)")
print(f"  Columns: {list(df_reviews.columns)}\n")

print("DATA TYPES:")
print(df_reviews.dtypes)
print()

print("HEAD (first 10 rows):")
print(df_reviews.head(10))
print()

print("TAIL (last 5 rows):")
print(df_reviews.tail(5))
print()

# Missing value analysis
print("MISSING VALUES (per column):")
missing_pct = (df_reviews.isnull().sum() / len(df_reviews) * 100)
for col, pct in missing_pct.items():
    print(f"  {col}: {df_reviews[col].isnull().sum():,} / {len(df_reviews):,} ({pct:.2f}%)")
print()

# Duplicates analysis
print("DUPLICATES ANALYSIS:")
full_row_dups = df_reviews.duplicated().sum()
print(f"  Full-row duplicates: {full_row_dups}")

# Check review_id (likely primary key)
if 'id' in df_reviews.columns:
    id_dups = df_reviews['id'].duplicated().sum()
    print(f"  Duplicates in 'id': {id_dups}")
print()

# Check for listing_id
if 'listing_id' in df_reviews.columns:
    print("LISTING_ID ANALYSIS:")
    print(f"  Unique listing_ids: {df_reviews['listing_id'].nunique():,}")
    print(f"  Null listing_ids: {df_reviews['listing_id'].isnull().sum():,}")
print()

# Review date analysis
print("REVIEW DATE ANALYSIS:")
date_cols = [col for col in df_reviews.columns if 'date' in col.lower() or col in ['Date', 'review_date']]
if date_cols:
    for date_col in date_cols:
        try:
            df_reviews[date_col] = pd.to_datetime(df_reviews[date_col], errors='coerce')
            print(f"  {date_col}:")
            print(f"    Range: {df_reviews[date_col].min()} to {df_reviews[date_col].max()}")
            print(f"    Null values: {df_reviews[date_col].isnull().sum()}")
        except:
            print(f"  {date_col}: Could not parse as datetime")
else:
    print("  No date columns detected")
print()

print("✓ Reviews EDA complete")


REVIEWS.CSV - EXPLORATORY DATA ANALYSIS

📁 File: reviews.csv
   Size: 336.53 MB

SHAPE & COLUMNS:
  Shape: (1275992, 6) (rows, columns)
  Columns: ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

DATA TYPES:
listing_id        Int64
id                Int64
date             string
reviewer_id       Int64
reviewer_name    string
comments         string
dtype: object

HEAD (first 10 rows):
   listing_id       id        date  reviewer_id reviewer_name  \
0      354838  1100092  2012-04-08       926074      Masahiko   
1      354838  2062149  2012-08-22      1915019     Jean-Marc   
2      354838  2333538  2012-09-18      2880602     Ekaterina   
3      354838  2412223  2012-09-25       751843         Ahmet   
4      354838  2466170  2012-10-01      3214589           Ani   
5      354838  2642713  2012-10-17      1741900        Andres   
6      354838  2927060  2012-11-20      3693269         Ayala   
7      354838  3185640  2012-12-30      1649127        Joseph   


## Task B1: Neighbourhoods Data Cleaning & Standardization

Normalize column names, trim strings, ensure consistent types, verify uniqueness, and save cleaned dataset.

In [16]:
print("\n" + "=" * 80)
print("NEIGHBOURHOODS DATA CLEANING")
print("=" * 80)

df_neigh_clean = df_neighbourhoods.copy()

# 1. Normalize column names to snake_case
df_neigh_clean.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df_neigh_clean.columns]
print(f"\n✓ Column names normalized: {list(df_neigh_clean.columns)}")

# 2. Trim string values and handle empty strings
for col in df_neigh_clean.columns:
    if df_neigh_clean[col].dtype == 'object':
        df_neigh_clean[col] = df_neigh_clean[col].str.strip()
        df_neigh_clean[col] = df_neigh_clean[col].replace('', pd.NA)
        print(f"  Trimmed strings in '{col}'")

# 3. Ensure consistent data types (IDs as int64 if numeric)
for col in df_neigh_clean.columns:
    if 'id' in col.lower():
        try:
            df_neigh_clean[col] = pd.to_numeric(df_neigh_clean[col], errors='coerce').astype('Int64')
            print(f"  Converted '{col}' to Int64")
        except:
            pass

# 4. Check for primary key uniqueness
print("\nPrimary Key Verification:")
for col in df_neigh_clean.columns:
    unique_count = df_neigh_clean[col].nunique()
    null_count = df_neigh_clean[col].isnull().sum()
    is_pk = (unique_count == len(df_neigh_clean)) and (null_count == 0)
    if is_pk:
        print(f"  ✓ '{col}' is a valid primary key ({unique_count} unique, {null_count} nulls)")

# 5. Final quality checks
print("\nFinal Quality Checks:")
assert df_neigh_clean.duplicated().sum() == 0, "ERROR: Duplicates found after cleaning!"
print(f"  ✓ No full-row duplicates")

assert df_neigh_clean.isnull().sum().sum() == 0 or df_neigh_clean.isnull().sum().sum() <= 1, \
    f"ERROR: More nulls than expected"
print(f"  ✓ Null values acceptable: {df_neigh_clean.isnull().sum().sum()}")

# 6. Save cleaned dataset
neigh_output_path = OUTPUT_DIR / "neighbourhoods_clean.parquet"
df_neigh_clean.to_parquet(neigh_output_path, index=False, compression='gzip')
print(f"\n✓ Cleaned neighbourhoods saved to: {neigh_output_path.name}")
print(f"  Shape: {df_neigh_clean.shape}")
print(f"  Size: {neigh_output_path.stat().st_size / (1024**2):.2f} MB")
print(f"\nCleaned neighbourhoods:")
print(df_neigh_clean)


NEIGHBOURHOODS DATA CLEANING

✓ Column names normalized: ['neighbourhood_group', 'neighbourhood']

Primary Key Verification:
  ✓ 'neighbourhood' is a valid primary key (128 unique, 0 nulls)

Final Quality Checks:
  ✓ No full-row duplicates
  ✓ Null values acceptable: 0

✓ Cleaned neighbourhoods saved to: neighbourhoods_clean.parquet
  Shape: (128, 2)
  Size: 0.00 MB

Cleaned neighbourhoods:
    neighbourhood_group  neighbourhood
0            Arganzuela        Acacias
1            Arganzuela         Atocha
2            Arganzuela        Chopera
3            Arganzuela       Delicias
4            Arganzuela       Imperial
..                  ...            ...
123          Villaverde       Butarque
124          Villaverde    Los Angeles
125          Villaverde    Los Rosales
126          Villaverde     San Andrés
127          Villaverde  San Cristobal

[128 rows x 2 columns]


## Task B2: Reviews Data Cleaning & Aggregation at Listing Level

Clean review records (normalize types, dates), then aggregate by listing_id to create review-derived features for joining with calendar_clean.

In [17]:
print("\n" + "=" * 80)
print("REVIEWS DATA CLEANING & AGGREGATION")
print("=" * 80)

df_reviews_clean = df_reviews.copy()

# 1. Normalize column names to snake_case
df_reviews_clean.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df_reviews_clean.columns]
print(f"\n✓ Column names normalized")

# 2. Trim string values
for col in df_reviews_clean.columns:
    if df_reviews_clean[col].dtype == 'object':
        try:
            df_reviews_clean[col] = df_reviews_clean[col].str.strip().replace('', pd.NA)
        except:
            pass

# 3. Convert review date column to datetime64[ns]
date_cols = [col for col in df_reviews_clean.columns if 'date' in col.lower()]
for date_col in date_cols:
    df_reviews_clean[date_col] = pd.to_datetime(df_reviews_clean[date_col], errors='coerce')
    print(f"  Converted '{date_col}' to datetime64[ns]")

# 4. Ensure listing_id is int64 with validation
if 'listing_id' in df_reviews_clean.columns:
    df_reviews_clean['listing_id'] = df_reviews_clean['listing_id'].astype('int64')
    assert df_reviews_clean['listing_id'].min() >= 0, "ERROR: Found negative listing_id values!"
    assert df_reviews_clean['listing_id'].isnull().sum() == 0, "ERROR: Found null listing_id values!"
    print(f"  Converted 'listing_id' to int64")
    print(f"    ✓ Min value: {df_reviews_clean['listing_id'].min()}")
    print(f"    ✓ Max value: {df_reviews_clean['listing_id'].max()}")

# 5. Save raw cleaned reviews for reference
reviews_output_path = OUTPUT_DIR / "reviews_clean.parquet"
df_reviews_clean.to_parquet(reviews_output_path, index=False, compression='gzip')
print(f"\n✓ Cleaned reviews saved to: {reviews_output_path.name}")
print(f"  Shape: {df_reviews_clean.shape}")
print(f"  Size: {reviews_output_path.stat().st_size / (1024**2):.2f} MB\n")

# 6. Aggregate reviews at listing level
print("-" * 80)
print("AGGREGATING REVIEWS AT LISTING LEVEL")
print("-" * 80)

if 'listing_id' in df_reviews_clean.columns:
    # Identify the date column for aggregation
    date_col = next((col for col in df_reviews_clean.columns if 'date' in col.lower()), None)
    
    agg_dict = {
        'id': 'count'  # Total count of reviews
    }
    
    # Add date-based aggregations if date column exists
    if date_col:
        agg_dict[date_col] = ['max', 'min']  # Last review date, first review date
    
    # Create aggregation
    reviews_listing_features = df_reviews_clean.groupby('listing_id').agg(agg_dict).reset_index()
    
    # Flatten column names
    if isinstance(reviews_listing_features.columns, pd.MultiIndex):
        reviews_listing_features.columns = ['_'.join(col).strip('_') for col in reviews_listing_features.columns]
    
    # Rename aggregated columns for clarity
    reviews_listing_features.rename(columns={'id_count': 'review_count_total'}, inplace=True)
    
    if date_col:
        reviews_listing_features.rename(columns={
            f'{date_col}_max': 'last_review_date',
            f'{date_col}_min': 'first_review_date'
        }, inplace=True)
    
    # Calculate temporal metrics if date column exists
    if date_col and 'last_review_date' in reviews_listing_features.columns:
        df_with_dates = reviews_listing_features.dropna(subset=['last_review_date'])
        
        # Only calculate if there are recent reviews
        if len(df_with_dates) > 0:
            today = pd.Timestamp.now()
            reviews_listing_features['days_since_last_review'] = \
                (today - reviews_listing_features['last_review_date']).dt.days
            
            # reviews_per_month = reviews_count / months_active
            reviews_listing_features['months_active'] = \
                (today - reviews_listing_features['first_review_date']).dt.days / 30.0
            
            reviews_listing_features['reviews_per_month'] = \
                reviews_listing_features['review_count_total'] / reviews_listing_features['months_active'].clip(lower=1)
            
            # AGGREGATION METRIC: reviews_90d (recent activity, proxy for demand)
            # Count reviews in last 90 days relative to calendar_clean.date.max()
            date_90d_ago = today - pd.Timedelta(days=90)
            reviews_90d = df_reviews_clean[
                (df_reviews_clean['listing_id'].notnull()) & 
                (df_reviews_clean[date_col] >= date_90d_ago)
            ].groupby('listing_id').size().reset_index(name='reviews_90d')
            
            reviews_listing_features = reviews_listing_features.merge(reviews_90d, on='listing_id', how='left')
            reviews_listing_features['reviews_90d'] = reviews_listing_features['reviews_90d'].fillna(0).astype('int64')
    
    # Ensure int64 for count columns
    for col in ['review_count_total', 'reviews_90d']:
        if col in reviews_listing_features.columns:
            reviews_listing_features[col] = reviews_listing_features[col].astype('int64')
    
    print(f"\n✓ Aggregation complete:")
    print(f"  Unique listings with reviews: {len(reviews_listing_features)}")
    print(f"  Columns created: {list(reviews_listing_features.columns)}")
    print(f"  Sample:\n{reviews_listing_features.head(10)}\n")
    
    # Save aggregated features
    features_output_path = OUTPUT_DIR / "reviews_listing_features.parquet"
    reviews_listing_features.to_parquet(features_output_path, index=False, compression='gzip')
    print(f"✓ Aggregated review features saved to: {features_output_path.name}")
    print(f"  Shape: {reviews_listing_features.shape}")
    print(f"  Size: {features_output_path.stat().st_size / (1024**2):.2f} MB")
    
    # Quality assertions
    print(f"\n✓ Quality Checks:")
    assert reviews_listing_features['review_count_total'].min() > 0, "ERROR: Found zero reviews!"
    assert reviews_listing_features['listing_id'].duplicated().sum() == 0, "ERROR: Duplicate listing_ids!"
    assert reviews_listing_features['listing_id'].isnull().sum() == 0, "ERROR: Null listing_ids!"
    print(f"  ✓ All review counts > 0")
    print(f"  ✓ No duplicate listing_ids")
    print(f"  ✓ No null listing_ids")
else:
    print("⚠️  WARNING: 'listing_id' column not found in reviews - skipping aggregation")


REVIEWS DATA CLEANING & AGGREGATION

✓ Column names normalized
  Converted 'date' to datetime64[ns]
  Converted 'listing_id' to int64
    ✓ Min value: 21853
    ✓ Max value: 1507409317740478788

✓ Cleaned reviews saved to: reviews_clean.parquet
  Shape: (1275992, 6)
  Size: 124.54 MB

--------------------------------------------------------------------------------
AGGREGATING REVIEWS AT LISTING LEVEL
--------------------------------------------------------------------------------

✓ Aggregation complete:
  Unique listings with reviews: 19853
  Columns created: ['listing_id', 'review_count_total', 'last_review_date', 'first_review_date', 'days_since_last_review', 'months_active', 'reviews_per_month', 'reviews_90d']
  Sample:
   listing_id  review_count_total last_review_date first_review_date  \
0       21853                  33       2018-07-15        2014-10-10   
1       30320                 173       2025-08-27        2010-07-06   
2       30959                   8       2017-05-3

## Task C1: Reviews Integration with Calendar_Clean

Load calendar_clean, join review-derived features by listing_id (memory-efficient left join), create calendar_enriched, and validate.

In [18]:
print("\n" + "=" * 80)
print("TASK C1: REVIEWS INTEGRATION WITH CALENDAR_CLEAN")
print("=" * 80)

# Load calendar_clean (try parquet first, then csv.gz)
calendar_clean_path = OUTPUT_DIR / "calendar_clean.parquet"
calendar_clean_csv_path = OUTPUT_DIR / "calendar_clean.csv.gz"

df_calendar = None

if calendar_clean_path.exists():
    print(f"\n📁 Loading {calendar_clean_path.name}...")
    df_calendar = pd.read_parquet(calendar_clean_path)
elif calendar_clean_csv_path.exists():
    print(f"\n📁 Loading {calendar_clean_csv_path.name}...")
    df_calendar = pd.read_csv(calendar_clean_csv_path)
    # Restore dtypes
    df_calendar['date'] = pd.to_datetime(df_calendar['date'])
    df_calendar['available'] = df_calendar['available'].astype('bool')
    df_calendar['listing_id'] = df_calendar['listing_id'].astype('int64')
    df_calendar['price'] = df_calendar['price'].astype('Float64')
else:
    print(f"⚠️  WARNING: calendar_clean files not found!")

if df_calendar is not None:
    print(f"  Shape: {df_calendar.shape}")
    print(f"  Columns: {list(df_calendar.columns)}")
    # Type validation
    assert df_calendar['listing_id'].dtype == np.int64, f"ERROR: listing_id dtype is {df_calendar['listing_id'].dtype}, expected int64!"
    print(f"  ✓ listing_id dtype verified: int64")

if df_calendar is not None and 'reviews_listing_features' in locals():
    print(f"\n📁 Loading {features_output_path.name}...")
    print(f"  Shape: {reviews_listing_features.shape}")
    print(f"  Columns: {list(reviews_listing_features.columns)}")
    
    # Memory check before join
    mem_cal_mb = df_calendar.memory_usage(deep=True).sum() / (1024**2)
    mem_features_mb = reviews_listing_features.memory_usage(deep=True).sum() / (1024**2)
    print(f"\n  Memory usage:")
    print(f"    calendar_clean: {mem_cal_mb:.2f} MB")
    print(f"    review_features: {mem_features_mb:.2f} MB")
    
    # Left join: calendar_clean on the left, review features on the right
    print(f"\n  Performing LEFT JOIN on 'listing_id'...")
    df_calendar_enriched = df_calendar.merge(
        reviews_listing_features,
        on='listing_id',
        how='left'
    )
    print(f"  ✓ Join complete")
    print(f"    Result shape: {df_calendar_enriched.shape}")
    
    # Validation: Check join integrity
    print(f"\n  JOIN VALIDATION:")
    
    # Check 1: Row count preserved (left join semantics)
    rows_matched = len(df_calendar_enriched)
    rows_original = len(df_calendar)
    assert rows_matched == rows_original, f"ERROR: Row count changed! {rows_original} → {rows_matched}"
    print(f"  ✓ Row count preserved: {rows_matched} == {rows_original}")
    
    # Check 2: No duplicates on (listing_id, date) - CRITICAL
    dups = df_calendar_enriched.duplicated(subset=['listing_id', 'date']).sum()
    assert dups == 0, f"ERROR: Found {dups} duplicates on (listing_id, date)!"
    print(f"  ✓ No duplicates on (listing_id, date): {dups}")
    
    # Check 3: Type validation on key columns
    assert df_calendar_enriched['listing_id'].dtype == np.int64, f"ERROR: listing_id dtype is {df_calendar_enriched['listing_id'].dtype}!"
    assert df_calendar_enriched['date'].dtype.name.startswith('datetime'), f"ERROR: date dtype is {df_calendar_enriched['date'].dtype}!"
    print(f"  ✓ Key column types valid (listing_id: int64, date: datetime)")
    
    # Check 4: Review feature columns have expected null patterns
    review_feature_cols = [col for col in df_calendar_enriched.columns 
                           if col.startswith('review_') or col in ['last_review_date', 'first_review_date', 'days_since_last_review']]
    if review_feature_cols:
        print(f"  ✓ Review feature columns created: {review_feature_cols}")
        for col in review_feature_cols:
            null_pct = df_calendar_enriched[col].isnull().sum() / len(df_calendar_enriched) * 100
            print(f"      {col}: {null_pct:.1f}% null (expected for listings without reviews)")
    
    # Check 5: Data types
    print(f"\n  DATA TYPES:")
    print(df_calendar_enriched.dtypes)
    
    # Save enriched calendar
    enriched_output_path = OUTPUT_DIR / "calendar_enriched.parquet"
    df_calendar_enriched.to_parquet(enriched_output_path, index=False, compression='gzip')
    enriched_size_mb = enriched_output_path.stat().st_size / (1024**2)
    print(f"\n✓ Calendar enriched saved to: {enriched_output_path.name}")
    print(f"  Shape: {df_calendar_enriched.shape}")
    print(f"  Size: {enriched_size_mb:.2f} MB ({(enriched_size_mb - mem_cal_mb):.2f} MB increase)")
    
    print(f"\n✓ Task C1 complete: calendar_enriched ready for analysis")
else:
    print("⚠️  Cannot proceed with integration: missing calendar_clean or review features")


TASK C1: REVIEWS INTEGRATION WITH CALENDAR_CLEAN

📁 Loading calendar_clean.csv.gz...
  Shape: (9125007, 6)
  Columns: ['listing_id', 'date', 'available', 'min_nights', 'max_nights', 'price']
  ✓ listing_id dtype verified: int64

📁 Loading reviews_listing_features.parquet...
  Shape: (19853, 8)
  Columns: ['listing_id', 'review_count_total', 'last_review_date', 'first_review_date', 'days_since_last_review', 'months_active', 'reviews_per_month', 'reviews_90d']

  Memory usage:
    calendar_clean: 365.50 MB
    review_features: 1.23 MB

  Performing LEFT JOIN on 'listing_id'...
  ✓ Join complete
    Result shape: (9125007, 13)

  JOIN VALIDATION:
  ✓ Row count preserved: 9125007 == 9125007
  ✓ No duplicates on (listing_id, date): 0
  ✓ Key column types valid (listing_id: int64, date: datetime)
  ✓ Review feature columns created: ['review_count_total', 'last_review_date', 'first_review_date', 'days_since_last_review']
      review_count_total: 20.6% null (expected for listings without rev

## Task C2: Neighbourhoods Integration Strategy

**IMPORTANT: neighbourhoods.csv contains only METADATA (names/IDs), NOT geometry polygons.**

Integration approach: Map via `listings.csv` neighbourhood field using string normalization. Spatial joins with polygon geometry require external GeoJSON/shapefile data.

In [19]:
print("\n" + "=" * 80)
print("TASK C2: NEIGHBOURHOODS INTEGRATION STRATEGY")
print("=" * 80)

print(f"\n📋 NEIGHBOURHOODS ANALYSIS:")
print(f"  Neighbourhoods dataset shape: {df_neigh_clean.shape}")
print(f"  Columns: {list(df_neigh_clean.columns)}")
print(f"  Sample:\n{df_neigh_clean.head()}\n")

# Strategy: Try to join neighbourhoods via listings.csv
listings_file = DATASET_DIR / "listings.csv"
if listings_file.exists():
    print(f"  📁 Attempting to load listings.csv to identify neighbourhood join key...")
    
    # Read listings with specific columns only (memory efficient)
    listings_cols_to_check = ['id', 'listing_id']
    listings_cols_to_check.extend([col for col in ['neighbourhood', 'neighbourhood_cleansed', 
                                                     'neighbourhood_group_cleansed', 'neighbourhood_group'] 
                                   if col in ['neighbourhood', 'neighbourhood_cleansed', 
                                             'neighbourhood_group_cleansed', 'neighbourhood_group']])
    
    try:
        # Sample listings to find join key
        df_listings_sample = pd.read_csv(listings_file, nrows=100)
        listing_cols = list(df_listings_sample.columns)
        print(f"    Listings columns: {listing_cols}")
        
        # Identify neighbourhood-related columns
        neigh_cols_in_listings = [col for col in listing_cols if 'neighbourhood' in col.lower()]
        neigh_names_in_listings = [col for col in listing_cols if 'neigh' in col.lower()]
        
        print(f"    Neighbourhood-related columns in listings: {neigh_cols_in_listings}")
        
        if neigh_cols_in_listings:
            print(f"\n  ✓ Neighbourhood columns found in listings!")
            print(f"    Potential join keys: {neigh_cols_in_listings}")
            
            # Try to find common neighbourhood names between listings and neighbourhoods
            neigh_names_listings = df_listings_sample[neigh_cols_in_listings[0]].unique()[:10]
            print(f"    Sample neighbourhood values in listings: {neigh_names_listings}")
            
            if len(df_neigh_clean) < 100:
                print(f"    Neighbourhoods dataset content:")
                print(df_neigh_clean)
            
            print(f"\n  ⚠️  NEIGHBOURHOODS INTEGRATION STRATEGY:")
            print(f"    1. Load full listings.csv with neighbourhood field")
            print(f"    2. Create (listing_id, neighbourhood) mapping with string normalization")
            print(f"    3. Merge mapping with calendar_enriched on listing_id")
            print(f"    4. Optional: merge neighbourhoods metadata on neighbourhood name")
            print(f"\n    NOTE: Attribute matching (not spatial) - string normalization required")
            
            # Load full listings for mapping (with memory consideration)
            print(f"\n  Loading full listings.csv for neighbourhood mapping...")
            df_listings = pd.read_csv(listings_file, dtype={'id': 'int64'})
            print(f"    Listings shape: {df_listings.shape}")
            
            neigh_col = neigh_cols_in_listings[0] if neigh_cols_in_listings else None
            if neigh_col:
                # Create listings-to-neighbourhood mapping
                listing_neighbourhood_map = df_listings[['id', neigh_col]].drop_duplicates()
                listing_neighbourhood_map.columns = ['listing_id', 'neighbourhood_listings']
                listing_neighbourhood_map['listing_id'] = listing_neighbourhood_map['listing_id'].astype('int64')
                
                # Normalize strings for matching
                listing_neighbourhood_map['neighbourhood_normalized'] = (
                    listing_neighbourhood_map['neighbourhood_listings']
                    .fillna('Unknown')
                    .str.lower()
                    .str.strip()
                )
                
                print(f"    ✓ Created mapping: {listing_neighbourhood_map.shape[0]:,} listings")
                print(f"    Unique neighbourhoods in listings: {listing_neighbourhood_map['neighbourhood_normalized'].nunique()}")
                
                # Try to join with calendar_enriched
                if 'df_calendar_enriched' in locals():
                    print(f"\n  Joining neighbourhoods with calendar_enriched...")
                    df_calendar_with_neigh = df_calendar_enriched.merge(
                        listing_neighbourhood_map[['listing_id', 'neighbourhood_normalized', 'neighbourhood_listings']],
                        on='listing_id',
                        how='left'
                    )
                    
                    print(f"    ✓ Join complete:")
                    print(f"      Result shape: {df_calendar_with_neigh.shape}")
                    print(f"      Null neighbourhoods: {df_calendar_with_neigh['neighbourhood_normalized'].isnull().sum()}")
                    
                    # Try to add neighbourhood metadata
                    if len(df_neigh_clean) > 0:
                        print(f"\n  Diagnostics: Matching neighbourhoods metadata...")
                        
                        # Normalize neighbourhoods dataset
                        df_neigh_clean_norm = df_neigh_clean.copy()
                        if df_neigh_clean_norm.shape[1] > 0:
                            text_col = [col for col in df_neigh_clean_norm.columns 
                                       if df_neigh_clean_norm[col].dtype == 'object'][0] if any(
                                           df_neigh_clean_norm[col].dtype == 'object' 
                                           for col in df_neigh_clean_norm.columns) else None
                            
                            if text_col:
                                df_neigh_clean_norm[f'{text_col}_normalized'] = (
                                    df_neigh_clean_norm[text_col].fillna('Unknown')
                                    .str.lower()
                                    .str.strip()
                                )
                                
                                matches = df_calendar_with_neigh['neighbourhood_normalized'].isin(
                                    df_neigh_clean_norm[f'{text_col}_normalized']
                                ).sum()
                                print(f"    Matches found: {matches} / {len(df_calendar_with_neigh):,} rows")
                    
                    # Save with neighbourhoods
                    neigh_enriched_path = OUTPUT_DIR / "calendar_enriched_with_neighbourhoods.parquet"
                    df_calendar_with_neigh.to_parquet(neigh_enriched_path, index=False, compression='gzip')
                    print(f"\n✓ Calendar with neighbourhoods saved to: {neigh_enriched_path.name}")
        else:
            print(f"  ⚠️  No clear neighbourhood column in listings")
            print(f"    Neighbourhoods dataset likely contains only metadata (requires spatial join with GeoJSON/shapefile)")
    except Exception as e:
        print(f"  ⚠️  Error processing listings: {str(e)[:100]}")
        print(f"    Neighbourhoods cannot be joined via attribute - spatial join needed (see section below)")
else:
    print(f"  ⚠️  listings.csv not found")
    print(f"  Neighbourhoods integration requires external listings data or spatial join capability")

print(f"\n" + "-" * 80)
print(f"STATUS: NEIGHBOURHOODS.CSV CONTENT ANALYSIS")
print(f"-" * 80)
print(f"""
✓ CONFIRMED: neighbourhoods.csv contains ATTRIBUTE METADATA ONLY (no geometry):
  - Format: Simple CSV table (128 neighbourhoods × 2 columns)
  - Columns: neighbourhood_group (admin district), neighbourhood (name)
  - Primary key: neighbourhood (name)
  - Use case: Lookup table for neighbourhood metadata

⚠️  LIMITATION: Cannot enable true spatial analysis without polygon boundaries:
  - No direct neighbourhood-level mapping (no geometry)
  - Requires intermediate mapping via listings.csv (attribute join)
  - String normalization essential (case/whitespace handling)

🔄 INTEGRATION STRATEGY:
  1. Create (listing_id → neighbourhood) mapping from listings.csv
  2. String normalization: .lower().strip() on neighbourhood names
  3. Left join mapping with calendar_enriched by listing_id
  4. Optional: merge neighbourhoods metadata for additional attributes

🗺️  FOR SPATIAL ANALYSIS (if needed):
  - Requires external GeoJSON/shapefile with neighbourhood polygons + coordinates
  - Then use GeoPandas spatial join (sjoin) on geometry
  - Future work: integrate spatial_data.geojson when available

📊 CURRENT OUTPUT: calendar_enriched_with_neighbourhoods.parquet
  - Contains neighbourhood names via attribute mapping
  - Ready for neighbourhood-level aggregations (not spatial operations)
""")


TASK C2: NEIGHBOURHOODS INTEGRATION STRATEGY

📋 NEIGHBOURHOODS ANALYSIS:
  Neighbourhoods dataset shape: (128, 2)
  Columns: ['neighbourhood_group', 'neighbourhood']
  Sample:
  neighbourhood_group neighbourhood
0          Arganzuela       Acacias
1          Arganzuela        Atocha
2          Arganzuela       Chopera
3          Arganzuela      Delicias
4          Arganzuela      Imperial

  📁 Attempting to load listings.csv to identify neighbourhood join key...
    Listings columns: ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourh

## Integration Quality Assurance & Final Summary

Comprehensive integrity checks on enriched datasets and summary of deliverables.

In [20]:
print("\n" + "=" * 80)
print("QUALITY ASSURANCE & FINAL SUMMARY")
print("=" * 80)

summary_report = {
    "datasets_processed": [],
    "artifacts_created": [],
    "integration_status": {},
    "recommendations": []
}

# Check cleaned datasets
print("\n📦 CLEANED DATASETS:")
if (OUTPUT_DIR / "neighbourhoods_clean.parquet").exists():
    df_neigh = pd.read_parquet(OUTPUT_DIR / "neighbourhoods_clean.parquet")
    print(f"  ✓ neighbourhoods_clean.parquet: {df_neigh.shape}")
    summary_report["datasets_processed"].append(("neighbourhoods_clean", df_neigh.shape))
    summary_report["artifacts_created"].append("neighbourhoods_clean.parquet")

if (OUTPUT_DIR / "reviews_clean.parquet").exists():
    df_rev = pd.read_parquet(OUTPUT_DIR / "reviews_clean.parquet")
    print(f"  ✓ reviews_clean.parquet: {df_rev.shape}")
    summary_report["datasets_processed"].append(("reviews_clean", df_rev.shape))
    summary_report["artifacts_created"].append("reviews_clean.parquet")

if (OUTPUT_DIR / "reviews_listing_features.parquet").exists():
    df_rev_feat = pd.read_parquet(OUTPUT_DIR / "reviews_listing_features.parquet")
    print(f"  ✓ reviews_listing_features.parquet: {df_rev_feat.shape}")
    summary_report["artifacts_created"].append("reviews_listing_features.parquet")

# Check enriched datasets
print("\n💾 ENRICHED DATASETS:")
if (OUTPUT_DIR / "calendar_enriched.parquet").exists():
    df_cal_enr = pd.read_parquet(OUTPUT_DIR / "calendar_enriched.parquet")
    print(f"  ✓ calendar_enriched.parquet: {df_cal_enr.shape}")
    print(f"    Columns added: {[col for col in df_cal_enr.columns if 'review' in col.lower() or col in ['last_review_date', 'first_review_date', 'days_since_last_review', 'reviews_per_month', 'reviews_90d']]}")
    summary_report["artifacts_created"].append("calendar_enriched.parquet")
    summary_report["integration_status"]["reviews"] = "✓ Integrated"

if (OUTPUT_DIR / "calendar_enriched_with_neighbourhoods.parquet").exists():
    df_cal_neigh = pd.read_parquet(OUTPUT_DIR / "calendar_enriched_with_neighbourhoods.parquet")
    print(f"  ✓ calendar_enriched_with_neighbourhoods.parquet: {df_cal_neigh.shape}")
    summary_report["artifacts_created"].append("calendar_enriched_with_neighbourhoods.parquet")
    summary_report["integration_status"]["neighbourhoods"] = "✓ Integrated via listings mapping"

# Final integrity assertions
if 'df_calendar_enriched' in locals():
    print(f"\n🔒 CRITICAL ASSERTIONS:")
    assert df_calendar_enriched.duplicated(subset=['listing_id', 'date']).sum() == 0, \
        "CRITICAL ERROR: Duplicates found on (listing_id, date)!"
    assert df_calendar_enriched['listing_id'].dtype == np.int64, \
        f"CRITICAL ERROR: listing_id dtype is {df_calendar_enriched['listing_id'].dtype}!"
    print(f"  ✓ No duplicates on (listing_id, date)")
    print(f"  ✓ listing_id is int64")

# Final recommendations
print("\n📋 RECOMMENDATIONS:")
summary_report["recommendations"].append(
    "Reviews: Use review_count_total, reviews_per_month, reviews_90d (last 90 days) as demand/activity proxies"
)
summary_report["recommendations"].append(
    "Neighbourhoods: Attribute mapping confirmed; spatial analysis requires external GeoJSON/shapefile"
)
summary_report["recommendations"].append(
    "For web-map: Use reviews_90d (recent activity) as marker size/color, availability_rate as heatmap"
)

print("\n".join([f"  • {r}" for r in summary_report["recommendations"]]))

print(f"\n✓ Quality Assurance complete")
print(f"\nFull Report:")
import json
print(json.dumps(summary_report, indent=2))


QUALITY ASSURANCE & FINAL SUMMARY

📦 CLEANED DATASETS:
  ✓ neighbourhoods_clean.parquet: (128, 2)
  ✓ reviews_clean.parquet: (1275992, 6)
  ✓ reviews_listing_features.parquet: (19853, 8)

💾 ENRICHED DATASETS:
  ✓ calendar_enriched.parquet: (9125007, 13)
    Columns added: ['review_count_total', 'last_review_date', 'first_review_date', 'days_since_last_review', 'reviews_per_month', 'reviews_90d']
  ✓ calendar_enriched_with_neighbourhoods.parquet: (9125007, 15)

🔒 CRITICAL ASSERTIONS:
  ✓ No duplicates on (listing_id, date)
  ✓ listing_id is int64

📋 RECOMMENDATIONS:
  • Reviews: Use review_count_total, reviews_per_month, reviews_90d (last 90 days) as demand/activity proxies
  • Neighbourhoods: Attribute mapping confirmed; spatial analysis requires external GeoJSON/shapefile
  • For web-map: Use reviews_90d (recent activity) as marker size/color, availability_rate as heatmap

✓ Quality Assurance complete

Full Report:
{
  "datasets_processed": [
    [
      "neighbourhoods_clean",
     

## Integration Summary: Research Value & Join Strategy

What each dataset adds:

- **neighbourhoods.csv**: Geographic/administrative context (neighbourhood names/IDs). Enables neighbourhood-level analysis (availability by district, review intensity by area).

- **reviews.csv**: Demand proxy and listing quality indicator. Aggregated at listing level: review counts, temporal patterns (last review date), and intensity (reviews/month, reviews in last 90 days).

Valid joins:

1. **Reviews → Calendar**: Recommended
   - `reviews.listing_id` → `calendar.listing_id` (left join)
   - Creates review-derived features at each calendar row
   - No data loss; listings without reviews get NA values
   - Result: `calendar_enriched.parquet` (9.1M rows + review columns)

2. **Neighbourhoods → Calendar**: Depends on data format
   - If neighbourhoods.csv contains attribute data (names/IDs from listings.csv):
     - Join via `listings.csv` neighbourhood field (string normalization needed)
     - Result: `calendar_enriched_with_neighbourhoods.parquet`
   - If neighbourhoods.csv contains polygon geometries (GeoJSON/shapefile):
     - Requires spatial join using GeoPandas (not implemented here)
     - Needs coordinates from listings.csv (latitude/longitude)

Limitations & Assumptions:

- **Reviews**: Aggregate per listing loses temporal patterns within a listing's history. Treated as static proxy for demand quality, not dynamic pricing signal.
- **Review intensity** (reviews_per_month): Proxy for popularity/demand, not actual booking volume (needs booking data).
- **Neighbourhoods without geometry**: Cannot enable true spatial analysis without polygon boundaries (e.g., mapping availability heatmaps by district).
- **Last review date**: Useful for identifying inactive listings, but review date distribution may be seasonal/biased.

Recommended downstream analyses:

1. Availability by review intensity (filter calendars by reviews_per_month quartiles, compare avg_availability)
2. Neighbourhood-level review statistics (mean reviews_per_month, review_count_total by district)
3. Seasonality patterns filtered by neighbourhood (lag availability by neighbourhood + season)
4. Interactive web-map: click neighbourhood → see availability + review stats + recent reviews

## Interactive Web-Map Suggestions

### Concept 1: Review Intensity & Availability Heatmap

Goal: Visualize listing availability colored by review intensity, with neighbourhood filtering.

Data source: `calendar_enriched.parquet` (aggregated to listing level: mean availability, review_per_month)

Interactive elements:
- Listing markers: colored by `reviews_per_month` (blue=low activity, red=high activity)
- Popup: neighbourhood, price, last_review_date, available_pct (last 30 days)
- Filter: dropdown for neighbourhood(s), slider for `review_count_total` (minimum)
- Heatmap toggle: switch between (a) review intensity overlay, (b) availability overlay

Libraries: Folium (Leaflet wrapper) or Plotly

---

### Concept 2: Neighbourhood Review & Availability Dashboard

Goal: Neighbourhood-level summary (mean review metrics, availability by month, review timeline)

Data aggregation (listing → neighbourhood → calendar row):
- Group by neighbourhood and time_window (month/week)
- Calculate: avg_availability, median_reviews_per_month, count_of_active_listings
- Compute: review_trend (reviews_90d / reviews_total ratio)

Interactive elements:
- Neighbourhood dropdown: select 1-5 neighbourhoods to compare
- Time range slider: filter to specific months
- Charts:
  - Time series: availability vs review_intensity over time
  - Box plot: distribution of reviews_per_month by neighbourhood
  - Bar chart: count of active listings per neighbourhood

Libraries: Plotly (interactive) + GeoPandas (if geometry available)

---

### Code Template (Folium Concept 1)

```python
import folium
import pandas as pd
from folium import plugins

# Load calendar_enriched + listings (for coordinates)
df_enriched = pd.read_parquet('data/processed/calendar_enriched.parquet')
df_listings = pd.read_csv('data/calendar.csv')  # or listings.csv with lat/lon

# Aggregate to listing level (mean over dates)
listing_stats = df_enriched.groupby('listing_id').agg({
    'available': 'mean',
    'review_count_total': 'first',
    'reviews_per_month': 'first',
    'last_review_date': 'first',
    'price': 'first'
}).reset_index()

# Merge with coordinates
df_map_data = listing_stats.merge(df_listings[['id', 'latitude', 'longitude', 'neighbourhood_cleansed']], 
                                   left_on='listing_id', right_on='id', how='left')

# Color scale: reviews_per_month
def color_reviews(val):
    if pd.isna(val): return 'gray'
    if val < 1: return 'blue'
    if val < 2: return 'green'
    if val < 3: return 'yellow'
    return 'red'

# Create map
m = folium.Map(location=[40.4168, -3.7038], zoom_start=12)  # Madrid center

# Add markers
for _, row in df_map_data.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"{row['neighbourhood_cleansed']}<br>Reviews/month: {row['reviews_per_month']:.2f}<br>Availability: {row['available']:.1%}",
            color=color_reviews(row['reviews_per_month']),
            fill=True,
            fillOpacity=0.7
        ).add_to(m)

# Save
m.save('data/processed/review_intensity_map.html')
```

---

### Next Steps for Interactive Filtering

To add dropdown & slider filtering, wrap map generation in Streamlit or Jupyter widgets:

```python
# Streamlit example:
import streamlit as st

selected_neighbourhood = st.selectbox('Neighbourhood', df_map_data['neighbourhood_cleansed'].unique())
min_reviews = st.slider('Min reviews/month', 0.0, 10.0, 0.0)

filtered_data = df_map_data[
    (df_map_data['neighbourhood_cleansed'] == selected_neighbourhood) &
    (df_map_data['reviews_per_month'] >= min_reviews)
]

# Generate map with filtered_data...
```

Implementation requires coordinate data from listings.csv